# Cohort Analysis for Subscription Business

## Import Libraries and CSV

In [6]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import locale
locale.setlocale(locale.LC_ALL, '')

orders = pd.read_csv('/data/notebook_files/Customer_Subscription_And_Transaction_Details.csv')

Link to data source: https://www.kaggle.com/datasets/sarathperingayi/performance-of-a-subscription-business

**Context**

This is a cohort analysis for a subscription based business that analyzes customer retention rates. This notebook takes 2 dates as input for the time frame you wish to analyze. The resulting table will tell you how many new customers from your timeframe continued their subscriptions, and at what level.

**Data Dictionary**

1. **cust_id:** A unique identifier for each customer

2. **transaction_type:** The type of transaction

3. **transaction_date:** The date when the transaction was made

4. **subscription_type:** The type of subscription, which can be BASIC, MAX, or PRO

5. **subscription_price:** The price of the subscription

6. **customer_gender:** The gender of the customer, which can be Male or Female

7. **age_group:** The age group of the customer

8. **customer_country:** The country where the customer is located

9. **referral_type:** The type of referral, which can be Google Ads, Facebook, Display, or others

## Required Input

In [7]:
start_date = pd.to_datetime('2022-01-01',utc=True).tz_localize(None)
end_date = pd.to_datetime('2022-01-30',utc=True).tz_localize(None)

## Data Exploration

In [8]:
orders.head()

,cust_id,transaction_type,transaction_date,subscription_type,subscription_price,customer_gender,age_group,customer_country,referral_type
0,1,initial,2020-09-01,BASIC,33,Male,35-44,Denmark,Google Ads
1,2,initial,2022-03-01,BASIC,53,Female,18-24,Denmark,facebook
2,3,initial,2020-10-01,MAX,99,Female,25-34,Norway,facebook
3,3,REDUCTION,2022-02-01,BASIC,53,Female,25-34,Norway,facebook
4,4,initial,2022-05-01,PRO,85,Male,18-24,Sweden,Display


### Date Range

In [9]:
orders['transaction_date'].min()

'2020-01-01'

In [10]:
orders['transaction_date'].max()

'2022-12-01'

### Data Types

In [11]:
orders.dtypes

cust_id int64
transaction_type object
transaction_date object
subscription_type object
subscription_price int64
customer_gender object
age_group object
customer_country object
referral_type object
dtype: object

## Converting 'transaction_date' Column to timedate formatting

In [12]:
orders['transaction_date'] = orders['transaction_date'].astype('datetime64')

## Check for duplicate data and remove (if any)

In [13]:
orders.duplicated().sum()

0

In [14]:
# removing duplicates if there were any
# orders = orders.loc[~orders.duplicated()]

## Subsetting Data Frame

In [15]:
orders = orders[['cust_id', 'transaction_type', 'transaction_date', 
           'subscription_type', 'subscription_price']]

## Checking NA Counts

In [16]:
orders.isna().sum()

cust_id 0
transaction_type 0
transaction_date 0
subscription_type 0
subscription_price 0
dtype: int64

In [17]:
# removing NA fields if any
# orders = orders.dropna

## Time Cohort

In [18]:
# All orders from time cohort
cohort_orders = orders[(orders['transaction_date'] >= start_date) & (orders['transaction_date'] <= end_date)]

## Filtering down to only new customers in cohort

In [19]:
new_customers_cohort = cohort_orders.query('`transaction_type` == "initial"')

In [20]:
# count of new customers in cohort
count_new_customers = len(new_customers_cohort)
count_new_customers

306

## Pulling all data for new customers in cohort

In [21]:
new_customer_list = new_customers_cohort['cust_id'].values.tolist()
new_customer_full_cohort = orders.query('cust_id==@new_customer_list')
new_customer_full_cohort 

,cust_id,transaction_type,transaction_date,subscription_type,subscription_price
91,55,initial,2022-01-01,BASIC,53
95,58,initial,2022-01-01,BASIC,53
96,59,initial,2022-01-01,MAX,119
97,59,REDUCTION,2022-11-01,BASIC,53
162,98,initial,2022-01-01,BASIC,53
...,...,...,...,...,...
17875,10227,initial,2022-01-01,MAX,119
17876,10227,REDUCTION,2022-08-01,BASIC,53
17907,10245,initial,2022-01-01,BASIC,53
17909,10247,initial,2022-01-01,PRO,85


## Counts

In [22]:
# number of new customers who go on to reduce their subscription
count_reduction = new_customer_full_cohort.query('`transaction_type` == "REDUCTION"')
count_reduction = count_reduction['cust_id'].nunique()
count_reduction

95

In [23]:
# number of new customers who renew their subscriptions at the same subscription level
count_churn = new_customer_full_cohort.query('`transaction_type` == "CHURN"')
count_churn = count_churn['cust_id'].nunique()
count_churn

33

In [24]:
# number of new customers who go on to upgrade their subscription
count_upgrade = new_customer_full_cohort.query('`transaction_type` == "UPGRADE"')
count_upgrade = count_upgrade['cust_id'].nunique()
count_upgrade

38

In [25]:
# creating percentage variables

count_reduction_percent = round((count_reduction/count_new_customers) * 100, 2) if count_new_customers else 'nan'
count_churn_percent = round((count_churn/count_new_customers) * 100, 2) if count_new_customers else 'nan'
count_upgrade_percent = round((count_upgrade/count_new_customers) * 100, 2) if count_new_customers else 'nan'

count_reduction_percent = 'nan' if count_reduction_percent == 'nan' else '{}%'.format(count_reduction_percent)
count_churn_percent = 'nan' if count_churn_percent == 'nan' else '{}%'.format(count_churn_percent)
count_upgrade_percent = 'nan' if count_upgrade_percent == 'nan' else '{}%'.format(count_upgrade_percent)

not_renewed = count_new_customers - (count_reduction + count_churn + count_upgrade) 

not_renewed_percent = round((not_renewed/count_new_customers) * 100, 2) if count_new_customers else 'nan'
not_renewed_percent = 'nan' if not_renewed_percent == 'nan' else '{}%'.format(not_renewed_percent)

## Final Analysis

In [26]:
print(f"""
Of the {count_new_customers} new customers from {start_date} to {end_date}, 
{count_churn_percent}, or {count_churn} customers, renewed their subscription at the same subscription level, 
{count_reduction_percent}, or {count_reduction} customers, renewed their subscription but reduced their subscription type,
and {count_upgrade_percent}, or {count_upgrade} customers, renewed their subscription and upgraded their subscription type.
This means {not_renewed_percent}, or {not_renewed} customers, have yet to renew their subscription.
""")

fig = go.Figure(data=[go.Table(header=dict(values=['','Customers', 'Churn', 'Reduction', 'Upgrade']),
                 cells=dict(values=[['Count', 'Percentage'], 
                                    [count_new_customers, '100%'],
                                    [count_churn, count_churn_percent],
                                    [count_reduction, count_reduction_percent],
                                    [count_upgrade, count_upgrade_percent]]))
                     ])
fig.update_layout(title_text='Cohort Analysis')
fig.show()


Of the 306 new customers from 2022-01-01 00:00:00 to 2022-01-30 00:00:00, 
10.78%, or 33 customers, renewed their subscription at the same subscription level, 
31.05%, or 95 customers, renewed their subscription but reduced their subscription type,
and 12.42%, or 38 customers, renewed their subscription and upgraded their subscription type.
This means 45.75%, or 140 customers, have yet to renew their subscription.

